In [1]:
# import required libraries 
import os
import warnings 
import numpy as np
import pandas as pd 
import yfinance as yf
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [2]:
def data_wrangle(path, dropped_columns=None):
    """A method that cleans the original dataset,
       restructures the dataset, and fills the missing values.
        
    Parameters
    ----------
    path : str
        Data path to the CSV file.
    dropped_columns : list, optional
        Columns to be dropped (default is None).
    
    Returns
    -------
    pd.DataFrame
        Cleaned and structured dataframe.
    """
    
    # Read the dataset from the given path
    df = pd.read_csv(path, header=None, names=["Date", "Time", "Open", "High", "Low", "Close", "Volume"])
    
    # Combine Date and Time columns into a single Date column
    df['Date'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    
    # Drop the now redundant Time column
    df.drop(columns=["Time"], inplace=True)
    
    # If there are any unnecessary columns specified, drop them
    if dropped_columns:
        df = df.drop(columns=dropped_columns)
    
    # Set the Date column as the index
    df.set_index('Date', inplace=True)
    
    # Return the cleaned dataframe
    return df

In [3]:
# load the dataset 
df_1hr = data_wrangle('data/XAUUSD_1hour.csv')
# check the dataset 
df_1hr.head(5)

,Open,High,Low,Close,Volume
Date,,,,,
2013-06-19 06:00:00,1366.57,1367.47,1366.34,1366.40,2539
2013-06-19 07:00:00,1366.40,1366.68,1364.91,1366.38,3728
2013-06-19 08:00:00,1366.41,1369.55,1365.62,1368.82,6879
2013-06-19 09:00:00,1368.82,1369.92,1367.77,1368.87,6454
2013-06-19 10:00:00,1368.87,1369.53,1367.93,1367.95,3803


In [4]:
df_3h = df_1hr.resample('3h', label='right', closed='right').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last'})
df_3h.dropna(inplace=True)
print(df_3h.isna().sum())
df_3h.to_csv('data/test.csv')

Open     0
High     0
Low      0
Close    0
dtype: int64


In [5]:
df_3h.head(5)

,Open,High,Low,Close
Date,,,,
2013-06-19 06:00:00,1366.57,1367.47,1366.34,1366.40
2013-06-19 09:00:00,1366.40,1369.92,1364.91,1368.87
2013-06-19 12:00:00,1368.87,1369.53,1366.10,1367.92
2013-06-19 15:00:00,1367.94,1375.83,1367.52,1373.43
2013-06-19 18:00:00,1373.44,1375.14,1372.16,1373.52


In [6]:
# load 1 min dataset 
df_1min = data_wrangle('data/XAUUSD_1min.csv')
df_1min.head(5)

,Open,High,Low,Close,Volume
Date,,,,,
2024-05-02 02:12:00,2324.07,2324.24,2323.87,2324.09,19
2024-05-02 02:13:00,2324.08,2324.42,2324.08,2324.39,25
2024-05-02 02:14:00,2324.34,2324.53,2324.19,2324.30,21
2024-05-02 02:15:00,2324.41,2324.51,2323.93,2324.33,30
2024-05-02 02:16:00,2324.43,2325.07,2324.33,2324.77,25


In [7]:
# resample 4 hours 
df_4h = df_1min.resample('4h', label='right', closed='right').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
# IG-MT4 excludes the last line 
# This is it
df_4h_ex = df_1min.resample('4h').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
df_4h.dropna(inplace=True)
df_4h_ex.dropna(inplace=True)
print(df_4h.isna().sum())
print(df_4h_ex.isna().sum())
df_4h.to_csv('data/4hr.csv')
df_4h_ex.to_csv('data/4hr_ex.csv')

Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64
Open      0
High      0
Low       0
Close     0
Volume    0
dtype: int64


In [8]:
def stochastic(df: pd.DataFrame, k_period: int = 9, d_period: int = 3) -> pd.DataFrame:
    """A stochastic function that calculates the Fast %K & Slow %D using EMA.
    
    Parameters
    ----------
    df: pd.DataFrame
        Input dataframe containing OHLC data.
    k_period: int, optional
        Period to calculate the Fast %K (default is 9).
    d_period: int, optional
        Period to calculate the Slow %D (default is 3).
    
    Returns
    -------
    pd.DataFrame
        DataFrame that contains Fast %K, Fast %D (EMA), and Slow %D (EMA).
    """

    # Find the highest high market price in the k period
    df['HighestHigh'] = df['High'].rolling(window=k_period).max()

    # Find the lowest low market price in the k period
    df['LowestLow'] = df['Low'].rolling(window=k_period).min()

    # Calculate Fast %K
    df['FastK'] = ((df['Close'] - df['LowestLow']) / (df['HighestHigh'] - df['LowestLow'])) * 100

    # Calculate Fast %D (EMA of Fast %K with period 1, which is just FastK itself)
    df['FastD'] = df['FastK']

    # Calculate Slow %D (EMA of Fast %D with period d_period)
    df['SlowD'] = df['FastD'].ewm(span=d_period, adjust=False).mean()

    # Drop temporary columns
    df.drop(columns=['HighestHigh', 'LowestLow'], inplace=True)

    # Return the dataframe with stochastic values
    return df